In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
np.random.seed(0)


import os
import wget
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
#data import
train=pd.read_csv('..\\data\\UCI-fault\\Faults.csv',header=None)
pd.set_option('display.max_columns', None)
train.columns = ["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",
              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",
              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",
              "Luminosity_Index", "SigmoidOfAreas", "Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults" ]
train

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,1687,1,0,80,0.0498,0.2415,0.1818,0.0047,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,1687,1,0,80,0.7647,0.3793,0.2069,0.0036,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,1623,1,0,100,0.9710,0.3426,0.3333,0.0037,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,1353,0,1,290,0.7287,0.4413,0.1556,0.0052,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,1353,0,1,185,0.0695,0.4486,0.0662,0.0126,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,249,277,325780,325796,273,54,22,35033,119,141,1360,0,1,40,0.3662,0.3906,0.5714,0.0206,0.5185,0.7273,0.0,2.4362,1.4472,1.2041,-0.4286,0.0026,0.7254,0,0,0,0,0,0,1
1937,144,175,340581,340598,287,44,24,34599,112,133,1360,0,1,40,0.2118,0.4554,0.5484,0.0228,0.7046,0.7083,0.0,2.4579,1.4914,1.2305,-0.4516,-0.0582,0.8173,0,0,0,0,0,0,1
1938,145,174,386779,386794,292,40,22,37572,120,140,1360,0,1,40,0.2132,0.3287,0.5172,0.0213,0.7250,0.6818,0.0,2.4654,1.4624,1.1761,-0.4828,0.0052,0.7079,0,0,0,0,0,0,1
1939,137,170,422497,422528,419,97,47,52715,117,140,1360,0,1,40,0.2015,0.5904,0.9394,0.0243,0.3402,0.6596,0.0,2.6222,1.5185,1.4914,-0.0606,-0.0171,0.9919,0,0,0,0,0,0,1


In [5]:
y_value = pd.DataFrame(train, columns=["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"])
x_value = pd.DataFrame(train, columns=["X_Minimum", "X_Maximum", "Y_Minimum", "Y_Maximum", "Pixels_Areas", "X_Perimeter", "Y_Perimeter",
              "Sum_of_Luminosity", "Minimum_of_Luminosity", "Maximum_of_Luminosity", "Length_of_Conveyer", "TypeOfSteel_A300",
              "TypeOfSteel_A400", "Steel_Plate_Thickness", "Edges_Index", "Empty_Index", "Square_Index", "Outside_X_Index",
              "Edges_X_Index", "Edges_Y_Index", "Outside_Global_Index", "LogOfAreas", "Log_X_Index", "Log_Y_Index", "Orientation_Index",
              "Luminosity_Index", "SigmoidOfAreas"])

#x, x_test, y, y_test = train_test_split(x_value,y_value, test_size=0.2, shuffle=True, random_state=1004)
#x_train, x_valid, y_train, y_valid = train_test_split(x,y, test_size=0.11, shuffle=True, random_state=1004)

tmp=[]
target_columns = ["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"]
for i in range(len(y_value)):
    for j in range(0,7):
        if y_value.loc[i][j] == 1:
            tmp.append(target_columns[j])
            #tmp.append(j)

print(type(tmp[0]))
targets = pd.DataFrame(tmp,columns=["target"])
y_value=pd.DataFrame(tmp)
#x_value['target'] = tmp
#x_value
'''
y_value=[]
tmp=np.array(tmp)
print(tmp)
for i in tmp:
    y_value.append(i)
'''



<class 'str'>


'\ny_value=[]\ntmp=np.array(tmp)\nprint(tmp)\nfor i in tmp:\n    y_value.append(i)\n'

In [11]:
train = train.drop(["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"],axis=1)
train['faults'] = tmp
train

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,faults
0,42,50,270900,270944,267,17,44,24220,76,108,1687,1,0,80,0.0498,0.2415,0.1818,0.0047,0.4706,1.0000,1.0,2.4265,0.9031,1.6435,0.8182,-0.2913,0.5822,Pastry
1,645,651,2538079,2538108,108,10,30,11397,84,123,1687,1,0,80,0.7647,0.3793,0.2069,0.0036,0.6000,0.9667,1.0,2.0334,0.7782,1.4624,0.7931,-0.1756,0.2984,Pastry
2,829,835,1553913,1553931,71,8,19,7972,99,125,1623,1,0,100,0.9710,0.3426,0.3333,0.0037,0.7500,0.9474,1.0,1.8513,0.7782,1.2553,0.6667,-0.1228,0.2150,Pastry
3,853,860,369370,369415,176,13,45,18996,99,126,1353,0,1,290,0.7287,0.4413,0.1556,0.0052,0.5385,1.0000,1.0,2.2455,0.8451,1.6532,0.8444,-0.1568,0.5212,Pastry
4,1289,1306,498078,498335,2409,60,260,246930,37,126,1353,0,1,185,0.0695,0.4486,0.0662,0.0126,0.2833,0.9885,1.0,3.3818,1.2305,2.4099,0.9338,-0.1992,1.0000,Pastry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936,249,277,325780,325796,273,54,22,35033,119,141,1360,0,1,40,0.3662,0.3906,0.5714,0.0206,0.5185,0.7273,0.0,2.4362,1.4472,1.2041,-0.4286,0.0026,0.7254,Other_Faults
1937,144,175,340581,340598,287,44,24,34599,112,133,1360,0,1,40,0.2118,0.4554,0.5484,0.0228,0.7046,0.7083,0.0,2.4579,1.4914,1.2305,-0.4516,-0.0582,0.8173,Other_Faults
1938,145,174,386779,386794,292,40,22,37572,120,140,1360,0,1,40,0.2132,0.3287,0.5172,0.0213,0.7250,0.6818,0.0,2.4654,1.4624,1.1761,-0.4828,0.0052,0.7079,Other_Faults
1939,137,170,422497,422528,419,97,47,52715,117,140,1360,0,1,40,0.2015,0.5904,0.9394,0.0243,0.3402,0.6596,0.0,2.6222,1.5185,1.4914,-0.0606,-0.0171,0.9919,Other_Faults


In [12]:
encoder=LabelEncoder()
categorical_dims={}

for col in train.columns:
    train[col]=encoder.fit_transform(train[col].values)
    categorical_dims[col]=train[col].nunique()
print(categorical_dims)

{'X_Minimum': 962, 'X_Maximum': 994, 'Y_Minimum': 1939, 'Y_Maximum': 1940, 'Pixels_Areas': 920, 'X_Perimeter': 399, 'Y_Perimeter': 317, 'Sum_of_Luminosity': 1909, 'Minimum_of_Luminosity': 161, 'Maximum_of_Luminosity': 100, 'Length_of_Conveyer': 84, 'TypeOfSteel_A300': 2, 'TypeOfSteel_A400': 2, 'Steel_Plate_Thickness': 24, 'Edges_Index': 1387, 'Empty_Index': 1338, 'Square_Index': 770, 'Outside_X_Index': 454, 'Edges_X_Index': 818, 'Edges_Y_Index': 648, 'Outside_Global_Index': 3, 'LogOfAreas': 914, 'Log_X_Index': 183, 'Log_Y_Index': 217, 'Orientation_Index': 918, 'Luminosity_Index': 1522, 'SigmoidOfAreas': 388, 'faults': 7}


In [14]:
features=train.columns.tolist()
features.remove('faults')
print(len(features))
print(features)
cat_idxs=[]
for i,x in enumerate(features):
    cat_idxs.append(i)


cat_dims=[]
for v in categorical_dims.values():
    cat_dims.append(v)

cat_dims=cat_dims[:-1]
print(cat_dims)
print(len(cat_dims))

27
['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum', 'Pixels_Areas', 'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index', 'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas', 'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index', 'SigmoidOfAreas']
[962, 994, 1939, 1940, 920, 399, 317, 1909, 161, 100, 84, 2, 2, 24, 1387, 1338, 770, 454, 818, 648, 3, 914, 183, 217, 918, 1522, 388]
27


In [23]:
tabnet_params = {
                 "cat_emb_dim":1,
                 "optimizer_fn":torch.optim.Adam,
                 "optimizer_params":dict(lr=2e-2),
                 "scheduler_params":{"step_size":50, # how to use learning rate scheduler
                                 "gamma":0.9},
                 "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                 "mask_type":'entmax' # "sparsemax"
                }

clf = TabNetClassifier(**tabnet_params
)

Device used : cuda


In [24]:
from sklearn.model_selection import train_test_split

X=train.drop(['faults'],axis=1)
Y=train['faults']
print(Y)

X=X.to_numpy()
Y=Y.to_numpy()

X_train, X_test, y_train, y_test=train_test_split(X,Y,test_size=0.1, shuffle=True, random_state=1004)
X_train, X_val, y_train, y_val=train_test_split(X_train,y_train,test_size=0.1, shuffle=True, random_state=1004)

print(X_train)

0       4
1       4
2       4
3       4
4       4
       ..
1936    3
1937    3
1938    3
1939    3
1940    3
Name: faults, Length: 1941, dtype: int32
[[  58   68  868 ...  621  363  251]
 [ 313  331 1102 ...  411 1324    9]
 [   2    6 1576 ...  473   28   40]
 ...
 [  38  169 1506 ...  243  633  387]
 [  40  198  983 ...  321  924  387]
 [  52   67 1293 ...   60  579   61]]


In [25]:
y_test

array([6, 0, 2, 3, 0, 3, 0, 4, 1, 3, 3, 3, 3, 3, 3, 2, 0, 0, 5, 3, 3, 2,
       5, 3, 3, 3, 4, 3, 3, 0, 2, 0, 3, 0, 2, 2, 0, 3, 2, 3, 6, 2, 2, 6,
       3, 0, 6, 4, 4, 0, 2, 4, 5, 0, 4, 4, 6, 6, 3, 3, 3, 2, 0, 0, 3, 3,
       0, 3, 6, 2, 3, 4, 3, 2, 3, 3, 0, 0, 0, 6, 0, 3, 0, 0, 2, 0, 3, 3,
       3, 0, 0, 4, 0, 2, 0, 3, 6, 0, 2, 3, 0, 4, 6, 0, 3, 0, 1, 3, 5, 3,
       2, 6, 0, 3, 3, 6, 2, 3, 6, 4, 6, 3, 0, 4, 0, 3, 5, 2, 0, 3, 3, 6,
       2, 0, 5, 0, 2, 3, 6, 2, 6, 4, 3, 0, 4, 0, 4, 6, 3, 2, 5, 4, 0, 1,
       3, 5, 2, 3, 3, 2, 2, 3, 2, 3, 3, 2, 2, 3, 0, 0, 0, 0, 3, 0, 3, 3,
       6, 3, 0, 6, 3, 2, 4, 2, 2, 2, 6, 3, 6, 2, 6, 0, 4, 3, 1])

In [26]:
#max_epochs = 100 if not os.getenv("CI", False) else 2

In [27]:
max_epochs = 100

# This illustrates the warm_start=False behaviour
#save_history = []

clf.fit(
        X_train=X_train, y_train=y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        eval_name=['train', 'valid'],
        eval_metric=['accuracy'],
        max_epochs=max_epochs , patience=20,
        batch_size=1024, virtual_batch_size=128,
        num_workers=0,
        weights=1,
        drop_last=False
    )
#save_history.append(clf.history["valid_auc"])
    
#assert(np.all(np.array(save_history[0]==np.array(save_history[1]))))

epoch 0  | loss: 2.79062 | train_accuracy: 0.08784 | valid_accuracy: 0.08    |  0:00:00s
epoch 1  | loss: 2.19894 | train_accuracy: 0.0732  | valid_accuracy: 0.08    |  0:00:00s
epoch 2  | loss: 1.91061 | train_accuracy: 0.07575 | valid_accuracy: 0.07429 |  0:00:00s
epoch 3  | loss: 1.70381 | train_accuracy: 0.08848 | valid_accuracy: 0.05714 |  0:00:00s
epoch 4  | loss: 1.5892  | train_accuracy: 0.09166 | valid_accuracy: 0.06857 |  0:00:00s
epoch 5  | loss: 1.5235  | train_accuracy: 0.13367 | valid_accuracy: 0.09714 |  0:00:00s
epoch 6  | loss: 1.44484 | train_accuracy: 0.14067 | valid_accuracy: 0.15429 |  0:00:00s
epoch 7  | loss: 1.37644 | train_accuracy: 0.15468 | valid_accuracy: 0.15429 |  0:00:00s
epoch 8  | loss: 1.27534 | train_accuracy: 0.16805 | valid_accuracy: 0.20571 |  0:00:00s
epoch 9  | loss: 1.18444 | train_accuracy: 0.18014 | valid_accuracy: 0.21714 |  0:00:00s
epoch 10 | loss: 1.13229 | train_accuracy: 0.18714 | valid_accuracy: 0.22857 |  0:00:01s
epoch 11 | loss: 1.09

In [28]:
pred = np.clip(clf.predict(X_test), a_min=0, a_max=None)

In [29]:
from sklearn.metrics import classification_report
report = classification_report(y_test, pred)
print(">> 모델 평가 리포트")
print(report)

>> 모델 평가 리포트
              precision    recall  f1-score   support

           0       0.62      0.52      0.56        46
           1       0.50      1.00      0.67         4
           2       0.86      0.94      0.90        34
           3       0.53      0.44      0.48        62
           4       0.33      0.72      0.46        18
           5       1.00      0.62      0.77         8
           6       0.88      0.61      0.72        23

    accuracy                           0.61       195
   macro avg       0.67      0.69      0.65       195
weighted avg       0.65      0.61      0.61       195

